<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="arial" color="#0099cc">
Web Page Categorization using Machine Learning
</font>
</h1>

<h2 align="left" style="line-height:200%;font-family:vazir;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Introduction and Problem Statement
</font>
</h2>


<h4 align="left">
<font face="Arial" >

In this project, our objective is to classify websites based on their title, content, and other relevant factors. This task falls under the domain of classification.
</font>
</h4>


<h2 align="left" style="line-height:200%;font-family:vazir;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Importing Essential Libraries
</font>
</h2>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from hazm import Normalizer, word_tokenize
import string
from stopwords_guilannlp import stopwords_output

<h2 align="left" style="line-height:200%;font-family:vazir;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Reading the Dataset and see some example
</font>
</h2>

In [2]:
train = pd.read_csv('../data/yektanet_train.csv')
train.head()

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...,عکس بیت‌کوین کش برای پروفایل,عکس بیت‌کوین کش برای پروفایل,عکس بیت کوین با کیفیت 4K عکس ارزهای دیجیتال عک...,jowhareh.com/photo/%D8%B9%DA%A9%D8%B3-%D8%A8%D...,jowhareh.com,3268
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی,آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات...,doctor-yab.ir/Search/14773/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,175
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...,دانلود تحریم‌گذر Geph برای اندروید,دانلود تحریم‌گذر Geph برای اندروید,دانلود نرم افزار Geph,palexe.site/dl/geph-android/,palexe.site,3402
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ,راه‌اندازی تلویزیون همگام‌سازی کنترل اتصال به ...,rokhdadeghtesadi.ir/43874/,rokhdadeghtesadi.ir,3811



<h4 style="text-align: left; font-family: Arial;">
  <strong>Deleting useless features. </strong>
</h4>

In [3]:
train.drop(['id','domain','url'], axis=1, inplace=True)


<h4 style="text-align: left; font-family: Arial;">
  <strong>Based on the following examples, we understand that the length of textt_content is very large and also contains some irrelevant information.so let's drop it</strong>
</h4>


In [4]:
for i in range(3,6):
    for column_name in train.columns:
        field = train.loc[i, column_name]
        print(f"{column_name}: {len(field)} : {field}")
    print('|||||||||||||||||||||||||||||||||')

category: 19 : تکنولوژی و کامپبوتر
description: 112 : نرم افزار Geph برای اندروید یک پلت‌فرم چندسکویی و فیلترشکنی با منبع باز است که می‌تواند دسترسی امن و مطمئن به اÛ
text_content: 1920 : دانلود تحریم‌گذر Geph برای اندروید خانه/اندروید/دانلود تحریم‌گذر Geph برای اندروید اپلیکیشناندرویدتحریم‌گذر دانلود تحریم‌گذر Geph برای اندروید شاهین دانلود نرم افزار Geph نمایش لینک‌های دانلود نرم افزار Geph برای اندروید یک پلت‌فرم چندسکویی و فیلترشکنی با منبع باز است که می‌تواند دسترسی امن و مطمئن به اینترنت آزاد و بدون سانسور را فراهم کند. به گفته‌ی سازندگان نرم افزار گف:«بر خلاف فیلترشکن‌ها و پراکسی‌های سنتی، Geph به طوری طراحی شده تا حتی در برابر سیستم‌های بسیار قدرتمند سانسور نیز مقاوم باشد.» مقر اصلی شرکت فیلترشکن Geph در استونی است. این کشور یکی از حوزه‌های قضایی اتحادیه اروپا است که قوانین محکمی برای حفظ حریم خصوصی دارد. دانلود تحریم‌گذر ۱.۱.۱.۱ برای اندروید دانلود تحریم‌گذر Orbot برای اندروید دانلود تحریم‌گذر Proton VPN برای اندروید – پروتون وی پی ان + پرمیوم کرک شده دانلود تحریم‌گذر وی پی ان

In [5]:
train.drop(['text_content'], axis=1, inplace=True)


<h2 align="left" style="line-height:200%;font-family:vazir;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Preprocessing the data.
</font>
</h2>


<h4 style="text-align: left; font-family: Arial;">
  <strong>The classes are highly unbalanced, so we balance them.</strong>
</h4>


In [6]:
train['category'].value_counts().count()

22

In [7]:
train['category'].value_counts()

سلامت                  614
ورزش                   514
حقوق و دولت و سیاست    486
هنر و سرگرمی           410
موسیقی                 314
تکنولوژی و کامپبوتر    287
تجارت و اقتصاد         283
فیلم و سینما           239
خودرو                  237
اجتماعی                209
سفر و گردشگری          182
غذا و نوشیدنی          171
مذهبی                  160
مسکن                   131
خانه و باغبانی         128
مد و زیبایی            118
کتاب و ادبیات           83
تحصیلات                 79
اشتغال                  47
علم و دانش              34
خانواده                 34
حیوانات خانگی           29
Name: category, dtype: int64

In [8]:
X_train = train.drop(columns=['category'])
y_train = train['category']

In [9]:
ros = RandomOverSampler(random_state=0)
data_resampled, targets_resampled = ros.fit_resample(X_train, y_train)

In [10]:
targets_resampled.value_counts()

کتاب و ادبیات          614
تجارت و اقتصاد         614
اشتغال                 614
خانواده                614
علم و دانش             614
تحصیلات                614
موسیقی                 614
خانه و باغبانی         614
هنر و سرگرمی           614
سفر و گردشگری          614
فیلم و سینما           614
مذهبی                  614
اجتماعی                614
مد و زیبایی            614
خودرو                  614
مسکن                   614
حقوق و دولت و سیاست    614
غذا و نوشیدنی          614
ورزش                   614
تکنولوژی و کامپبوتر    614
سلامت                  614
حیوانات خانگی          614
Name: category, dtype: int64


<h4 style="text-align: left; font-family: Arial;">
  <strong>hazm is a popular Persian language processing (NLP) library that provides tools for tokenization, stemming, and normalization of Persian text.
  We will use the hazm library and related libraries for preprocessing.</strong>
</h4>


In [11]:
persian_stopwords = stopwords_output("Persian", "nar")

In [12]:
normalizer = Normalizer()
def preprocessor(input):
    punc_removed = input.translate(str.maketrans('', '', string.punctuation))
    normalized = normalizer.normalize(punc_removed)
    tokens = word_tokenize(normalized)
    filtered = []
    for token in tokens:
        token = str(token)
        token = token.lower()
        if not token in persian_stopwords and not token.isdigit():
            filtered.append(token)
    output = ' '.join(filtered)
    return output

In [13]:
def tokenizer(text):
    return word_tokenize(text)


<h4 style="text-align: left; font-family: Arial;">
  <strong>Example of preprocessor and word_tokenize</strong>
</h4>

In [14]:
print(train.h2[2])
print(preprocessor(train.h2[2]))
print(tokenizer(preprocessor(train.h2[2])))

آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات در مورد دکتر مهناز عابدینی
آدرس تلفن دکتر مهناز عابدینی نظرات تجربیات دکتر مهناز عابدینی
['آدرس', 'تلفن', 'دکتر', 'مهناز', 'عابدینی', 'نظرات', 'تجربیات', 'دکتر', 'مهناز', 'عابدینی']



<h2 align="left" style="line-height:200%;font-family:vazir;font-family:Arial;color:#0099cc">
<font face="Arial" color="#0099cc">
Let's build our mode
</font>
</h2>


<h4 style="text-align: left; font-family: Arial;">
  <strong>Trying Different Algorithms to Achieve Desired Results</strong>
</h4>


In [15]:
classifiers = [
    ('LinearSVC', LinearSVC(loss='hinge', penalty='l2', max_iter=5000)),
    ('RandomForest', RandomForestClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('LogisticRegression', LogisticRegression(max_iter=5000)),
    ('KNeighbors', KNeighborsClassifier()),
    ('MultinomialNB', MultinomialNB()),
    ('DecisionTree', DecisionTreeClassifier()),
    ('MLPClassifier', MLPClassifier(max_iter=500))
]

class ColumnConcatenator(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns[0]].astype(str) + ' ' + X[self.columns[1]].astype(str) +X[self.columns[2]].astype(str)+X[self.columns[3]].astype(str)
encoder = LabelEncoder()
y_train = encoder.fit_transform(targets_resampled)

X_train, X_test, y_train, y_test = train_test_split(data_resampled[['title', 'h1','description','h2']], targets_resampled, test_size=0.1, random_state=42)
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)

best_model = None
max_f1 = 0
for clf_name, clf in classifiers:

    pipeline = Pipeline([
        ('concat', ColumnConcatenator(columns=['title', 'h1','description','h2'])),
        ('vect', CountVectorizer(tokenizer=tokenizer, preprocessor=preprocessor,
                                analyzer='word', ngram_range=(1, 2),
                                min_df=5, lowercase=False)),
        ('tfidf', TfidfTransformer(sublinear_tf=True)),
        ('clf', clf) 
    ])
    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    if f1> max_f1:
        best_model = model
    print(f"Classifier: {clf_name}")
    print(f"F1 Score: {f1:.4f}")
    print()

c:\Users\Asus\Desktop\test\envq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classifier: LinearSVC
F1 Score: 0.9670



c:\Users\Asus\Desktop\test\envq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classifier: RandomForest
F1 Score: 0.9588



c:\Users\Asus\Desktop\test\envq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

Classifier: MultinomialNB
F1 Score: 0.9166



c:\Users\Asus\Desktop\test\envq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classifier: DecisionTree
F1 Score: 0.9419



c:\Users\Asus\Desktop\test\envq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classifier: MLPClassifier
F1 Score: 0.9693



In [16]:
best_model

Pipeline(steps=[('concat',
                 ColumnConcatenator(columns=['title', 'h1', 'description',
                                             'h2'])),
                ('vect',
                 CountVectorizer(lowercase=False, min_df=5, ngram_range=(1, 2),
                                 preprocessor=<function preprocessor at 0x0000013238807430>,
                                 tokenizer=<function tokenizer at 0x00000132387773A0>)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', MLPClassifier(max_iter=500))])

<h1  style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="arial" color="#0099cc">
MLPClassifier provides us with very high accuracy.
</font>
</h1>

In [17]:
predictions_train = best_model.predict(X_train)
print('train F1-score:', f1_score(y_train, predictions_train, average='weighted'))
predictions_test = best_model.predict(X_test)
print('test F1-score:', f1_score(y_test, predictions_test, average='weighted'))

train F1-score: 0.9981904761869375
test F1-score: 0.9693200399865424


<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="arial" color="#0099cc">
Thank you for your attention
</font>
</h1>